In [100]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [101]:
result_list = []
for num in range(1, 530):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
    params = {
        'page' : num
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content)
    tr_list = soup.find_all('tr')[1:]
    for tr in tr_list:
        time = tr.find_all('td')[0].text.strip()
        a = time.split('.')
        time2 = date(int(a[0]), int(a[1]), int(a[2]))
        price = tr.find_all('td')[1].text
        
        new_dict = {
            'date' : time2,
            'price' : price
        }
        result_list.append(new_dict)

In [175]:
df = pd.DataFrame(result_list)
df = df.set_index('date')
df.head()

,price
date,
2019-07-15,1.73
2019-07-12,1.74
2019-07-11,1.75
2019-07-10,1.74
2019-07-09,1.69


# 엑셀파이로 저장

In [104]:
import os

In [105]:
BASE_DIR = os.getcwd()
SAVE_DB_DIR = os.path.join(BASE_DIR, 'call')

if not os.path.exists(SAVE_DB_DIR):
    os.makedirs(SAVE_DB_DIR)

#텍스트 파일 만들기
temp_name = f"call_040816.xlsx"
txt_name = os.path.join(SAVE_DB_DIR, temp_name)
excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
#     df_report.to_excel(excel_writer, sheet_name='report')
df.to_excel(excel_writer)

excel_writer.save()

In [53]:
df[df['date'] == date(2018, 7, 6)].

array([False])

# 콜금리 상승/ 하락 판단  
 - 1개월전 금리와 비교하여 판단
 - 해당일이 존재하지 않을 경우 전날 가격으로 판단

### 판다스 DateOffset 모듈  
 - 알아서 다 해준다.

In [133]:
day = date(2018, 1, 3)
a = day - pd.DateOffset(months = 1)
a

Timestamp('2017-12-03 00:00:00')

In [141]:
df.loc[date(2005, 1, 3)]

price    3.27
Name: 2005-01-03, dtype: object

In [152]:
day = date(2005, 1, 1)
b = day - pd.DateOffset(days = 1)
print(b)
print(b.date())
df.loc[b.date()].price

2004-12-31 00:00:00
2004-12-31


'3.29'

In [222]:
def up_down_call():
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)
                
            if time2.date() in df.index:

                if df.loc[time].price >= df.loc[time2.date()].price:
                    df_ud['up_down'][i] = 1
                else:
                    df_ud['up_down'][i] = 0

    return df_ud

In [231]:
df_t = up_down_call()
df_t.head()

,up_down
date,
2019-07-15,0
2019-07-12,0
2019-07-11,1
2019-07-10,0
2019-07-09,0


In [237]:
df_a = df_t[:date(2005, 1, 1)]
df_a.tail()

,up_down
date,
2005-01-07,1
2005-01-06,0
2005-01-05,1
2005-01-04,1
2005-01-03,1
